In [4]:
search_for = 123
start_from = 0
threads = 10
thread = 1
log_freq = 10000



#in this cell, for a given lattice, we compute all forbidden nodes, 
#i.e. those nodes whose Voronoi cell is at a distance less than doubled covering radius

#An* represented in m=n+1 as sum x_i=0 with x_i\equiv to each other mod m. 
#Forbidden nodes will be recorded w.r.t the basis (-n,1,1,..) and permutations

n=5 #lattice dim
m=n+1 #space dim




R = sqrt(n*(n+1)*(n+2)/12) #covering radius

#first we generate all nodes with norm <=4R, and with norm<=2R
t=1 #.3 #can increase a bit for random from non-forbidden
short = []
vshort = []
K = ceil(t*4*R)
K2 = (t*4*R)^2
Kv = (2*R)^2

indexes = [] #arrays of integers between -K and K having the same residue modulo m 
for rem in range(m):
    res = []
    for i in range(-K,K+1):
        if i%m==rem:
            res.append(i)
    indexes.append(res)

for c in range(0,K+1):
    for coefs in mrange_iter([indexes[c%m] for i in range(m-2)]):
        p = [c]+list(coefs)+[-c-sum(coefs)]
        is_sorted = True
        for i in range(m-1):
            if p[i]<p[i+1]:
                is_sorted = False
                break
        if is_sorted:
            norm2 = sum((p[i])^2 for i in range(m))
            if norm2>0:
                if norm2 <= Kv:
                    vshort.append(p)
                if norm2 <= K2:
                    short.append(p)           

print("very short:",len(vshort))
print("short:",len(short))


from time import time
tm = time()


#defining the cone arising due to symmetry - it suffices to construct Voronoi cell in it
ineqs = [] 
for i in range(m-1):
    toadd = [0]*(m+1)
    toadd[i+1]=1
    toadd[i+2]=-1
    ineqs.append(toadd) 
    
ineqs += [[0]+[1]*m,[0]+[-1]*m] #we are in the hyperplane where all coords sum to zero

#now Voronoi cell planes
for p in vshort: #suffices to look at nodes at most 2R
    ineqs.append([sum((p[i])^2 for i in range(m))]+[-2*p[i] for i in range(m)])

V = Polyhedron(ieqs=ineqs)
    
print(time()-tm)    

### orthogonal projection affine operator onto (improper) non-empty polytope 
zerom = matrix(m)
def projp(P):
    nv = P.n_vertices()
    z = vector(P.vertices()[0])
    if nv==1:
        return zerom,z
    A = matrix([vector(P.vertices()[i])-z for i in range(1,nv)]).transpose()
    return A*A.pseudoinverse(),z


forb = []

for d in range(0,n): #dim of face
    for ff in V.faces(d):
        proj = projp(ff)
        for p in short:
            if p not in forb:
                x = vector([t/2 for t in p]) #dist from 1/2 of a node to the Voronoi cell equals 1/2 of the dist between cells at zero and at the node
                proj_pnt = proj[0]*(x-proj[1])+proj[1]
                if proj_pnt in ff.as_polyhedron():
                    dist = (x-proj_pnt).norm()
                    if dist<R:
                        forb.append(p)

print("forbidden:",len(forb))

#expand by the basis (-n,1,1,..) and permutations
def basis_coefs(x):
    y = x[0:(m-1)]
    sy = sum(y)
    return [(-sy-y[i])/m for i in range(m-1)]

### forbd = forb with possible permutations
forbd = {}
forbdv = []
for x in forb:
    for t in Permutations(list(x)):
        c = vector(basis_coefs(t))
        if str(c) not in forbd:
            forbd[str(c)]=0
            if sum(c)>=0:
                forbdv.append(c)
print("forbidden without symmetries:",len(forbd))
###





fn = 0
for x in forb:
    norm2 = sum(t^2 for t in x)
    if norm2>fn:
        fn = norm2
print("square L2 norm of forbidden:",fn)


small = 2 #max abs value of a small coefficient
sc = []
for c in mrange_iter([range(-small,small+1) for i in range(n)]):
    if sum(c)>=0:
        sc.append(vector(c))
sc.sort(key=lambda i: sum(x*x for x in i)) #abs(x) in place of x*x seems a bit slower
sc = sc[1:] #removing zero

print("small:",len(sc))

order = [(-1)^i*(1/4+i/2)-1/4 for i in range(1000)] #[0,-1,1,-2,2,...]

def check_sublattice(basis,target=Infinity): 
    A = matrix(basis) #rows
    #A = A.LLL() #benchmark to see if helps
    A = A.transpose() #columns

#     Ad = abs(A.det())
#     if Ad==0 or Ad>=target:
#         return Infinity
        
    for coefs in sc:
        if str(A*coefs) in forbd:
            return Infinity

    Ai = A.inverse()
    Ai_norms = [sum(t^2 for t in Ai[i]) for i in range(n)]
    M = []
    Mp = 1
    Mps = 1
    for i in range(n):
        mi = floor(sqrt(fn*Ai_norms[i]))
        M.append(mi)
        Mp *= (2*mi+1)
        Mps *= (2*min(small,mi)+1)
        
    if max(M)>small: #otherwise already considered
        if Mp-Mps<len(forbd): #faster using A, otherwise A_inv
            for coefs in mrange_iter([order[0:(2*M[i]+1)] for i in range(n)]):
                if sum(coefs)>=0 and max(abs(x) for x in coefs)>small:
                    if str(A*vector(coefs)) in forbd:
                        return Infinity
        else: #check if A_inv times forbidden is not in Z^n
            for x in forbdv:
                y = Ai*x
                allint = True
                for t in y:
                    if t!=floor(t):
                        allint = False
                        break
                if allint:
                    return Infinity   
    
    return 0#Ad



#random/exhaustive sublattice search, skip to the cell starting with good for search using non-forbidden generating vectors of small norm
#each type of a matrix is the numbers on the diagonal
maxd = search_for #bound for diagonal entries
mindet = search_for #minimum determinant
maxdet = search_for #maximum determinant

def all_types(m,d1,d2,n):
    res = []
    if n==1:
        for k in range(d1,min(d2,m)+1):
            res.append([k])
        return res
    for k in range(1,min(d2,m)+1):
        prev = all_types(m,ceil(d1/k),floor(d2/k),n-1)
        for x in prev:
            res.append([k]+x)
    return res

types = all_types(maxd,mindet,maxdet,n)

print("types:",len(types))

def ind_max(t): #given type, lists maxima of indexes to define entries above the diagonal
    return [t[i]^i for i in range(1,n)]


allc = 0
for t in types:
    allc += prod(ind_max(t))
print("all cases:",allc)

from random import randint
def rand_ind(t): #given type, return random indexes
    maxs = ind_max(t)
    return [randint(0,maxs[i]-1) for i in range(n-1)]

def sublat_mat(t,ind): #given type and indexes, return sublattice basis
    res = matrix(n)
    for i in range(n):
        res[i,i] = t[i]
    for i in range(1,n):
        tt = -floor(t[i]/2)
        indc = ind[i-1]
        for j in range(i):
            res[j,i] = tt + (indc % t[i])
            indc = floor(indc / t[i])
    return res

def exh_search():
    tm = time()
    bestv = Infinity
    best = []
    allcnt = 0
    for tt in types:
        print("type:",tt,prod(tt))
        cnt = 0
        maxinds = ind_max(tt)
        cM = prod(maxinds)
        print("cases of this type:",cM)
        for i in range(cM):
            allcnt += 1
            if allcnt%(log_freq*threads)==0:
                print(allcnt,(time()-tm)/3600*(allc-allcnt)/(log_freq*threads))
                tm=time()
            if allcnt>=start_from and allcnt%threads==thread:
                inds = []
                l = i
                for j in range(n-1):
                    inds.append(l%maxinds[j])
                    l=floor(l/maxinds[j])
                mat = sublat_mat(tt,inds)
                cs = check_sublattice(mat)
                if cs==0:
                    cs = prod(tt)
                    if cs<bestv:
                        print("sublattice basis:")
                        print(mat)
                        bestv = cs
                        best = [mat]
                    elif cs==bestv:
                        best.append(mat)
        print("best so far:",len(best))
    print("all cases:",allcnt)
    print("best:",len(best))
    return best





best = exh_search()




very short: 5
short: 39
0.0042111873626708984
forbidden: 31
forbidden without symmetries: 1984
square L2 norm of forbidden: 264
small: 1752
types: 25
all cases: 350465005
type: [1, 1, 1, 1, 123] 123
cases of this type: 228886641
100000 17.14346455261534
200000 25.19483432513422
300000 25.347593860972495
400000 25.209350828665844
500000 26.535396715700283
600000 24.729362111373497
700000 24.735257134172944
800000 24.779543647441166
900000 23.58005569873346
1000000 10.662615556893915
1100000 24.20951214915212
1200000 24.658642875703627
1300000 24.32074844005392
1400000 25.100832156011112
1500000 25.291675151445713
1600000 24.464473046874225
1700000 24.86999028395474
1800000 25.006634969692175
1900000 9.953150617298395
2000000 25.358224951882722
2100000 29.322188773584937
2200000 28.811010560942965
2300000 28.917201950296434
2400000 29.3540633440832
2500000 27.58942069507886
2600000 27.755572375484192
2700000 26.94715955608085
2800000 17.313487894775093
2900000 21.882468345468055
3000000 

29400000 23.4231385189282
29500000 23.71150339944005
29600000 12.266232433250586
29700000 23.274755420503187
29800000 24.02018759703773
29900000 24.26102055479118
30000000 22.360454388180585
30100000 12.964442560888717
30200000 24.43529484205041
30300000 23.935732939623296
30400000 23.621923850703844
30500000 22.49117634725005
30600000 21.39500416862601
30700000 16.31313432625133
30800000 14.97231456293755
30900000 21.392370212704837
31000000 22.83548614220086
31100000 23.138775723144516
31200000 23.89841061215088
31300000 24.537499023147276
31400000 15.433737552824681
31500000 21.2691996301854
31600000 23.93149999036038
31700000 24.7730489413897
31800000 23.843347797322142
31900000 17.645225649027932
32000000 18.30136501388057
32100000 24.378432677777727
32200000 22.6764708499025
32300000 23.522336412964425
32400000 21.781344601874682
32500000 20.59140830465257
32600000 8.669356251679908
32700000 21.91232925946703
32800000 20.85033801585572
32900000 22.556188082981603
33000000 22.4363

59100000 11.185372677260922
59200000 19.80350362578536
59300000 20.279509256410226
59400000 20.75522970680164
59500000 21.103566003031133
59600000 22.442271201942262
59700000 21.493923670240395
59800000 21.29108736146209
59900000 20.36652726985895
60000000 21.322525783680334
60100000 9.272153309530148
60200000 19.205327981361126
60300000 19.083567481005623
60400000 19.53192504378
60500000 9.62405164188415
60600000 17.511916800247793
60700000 19.48540759926499
60800000 18.81314716194443
60900000 17.032026797169618
61000000 13.564532237788987
61100000 20.265939323129494
61200000 21.188277229573295
61300000 20.912044990362794
61400000 21.358788445389898
61500000 21.5331984706056
61600000 21.39830584004787
61700000 21.090894511305905
61800000 19.985904102107895
61900000 17.176141175012212
62000000 12.287384027860009
62100000 18.541342918597955
62200000 19.77786827812481
62300000 19.03412197500956
62400000 7.747151900312046
62500000 19.07626280072567
62600000 18.874622667921212
62700000 19.

88900000 17.437109206941646
89000000 17.134142939746237
89100000 17.478720525673687
89200000 18.648051130209208
89300000 19.055586530425884
89400000 19.25110416846692
89500000 19.084306946698934
89600000 18.39099675655205
89700000 17.391176473133825
89800000 17.402460179401487
89900000 18.10472487739444
90000000 16.80824192537056
90100000 8.466999129083902
90200000 17.126899144799026
90300000 6.54624225298953
90400000 17.8355803628968
90500000 9.014908448186548
90600000 17.82071808918498
90700000 17.651551904155433
90800000 17.616602571972113
90900000 16.76292447724539
91000000 17.56812278825444
91100000 18.913057944249626
91200000 19.101329842148264
91300000 19.710656688294154
91400000 18.72445497635237
91500000 17.293286066151687
91600000 16.785826185352494
91700000 17.488077972934295
91800000 17.333975398300677
91900000 16.402785213314207
92000000 8.767660871272515
92100000 13.847221780073685
92200000 10.455449752224526
92300000 11.58177985601373
92400000 14.054914321634199
92500000

117900000 1.7915461087219453
118000000 1.785826713474691
118100000 1.7980325962369244
118200000 1.8016442943163975
118300000 1.7249756332896056
118400000 1.7177619569558402
118500000 1.721351191222459
118600000 1.734551734651903
118700000 1.8003208965430457
118800000 1.7819619877984025
118900000 1.7985973998771312
119000000 1.7818118156742817
119100000 2.6805232052399255
119200000 23.31932392457758
119300000 21.926660043617122
119400000 21.248807257769517
119500000 21.236319789727148
119600000 20.954047690159246
119700000 20.77274674956316
119800000 20.351410225910513
119900000 21.046292505924146
120000000 10.780757802429616
120100000 2.692910924278835
120200000 19.652677196265234
120300000 19.980793453606704
120400000 20.62181820632992
120500000 20.24851384838645
120600000 21.79662382955624
120700000 21.084355956168025
120800000 21.7028329652676
120900000 22.15171016726059
121000000 21.29428559292312
121100000 17.99964368871107
121200000 17.549969260563028
121300000 16.84502690751189


...

232900000 2.260886640783022
233000000 2.970238511611415
233100000 2.246422137807316
233200000 2.153534658447713
233300000 2.021464538442999
233400000 2.3723620676121384
233500000 1.1983057006048559
233600000 0.7672634099867444
233700000 1.5137789882966841
233800000 2.008496724675715
233900000 1.9272075879821877
234000000 2.0938171903358302
234100000 3.813382620577544
234200000 2.6481259468779696
234300000 2.054084087152064
234400000 2.049095966228992
234500000 2.20581864476991
234600000 2.3076415613628725
234700000 1.8608928424618343
234800000 2.07066939995211
234900000 2.1060543860802685
235000000 2.1000603500695227
235100000 2.791232251819346
235200000 3.3035722237896987
235300000 1.9267130405786532
235400000 0.5171029872289101
235500000 0.5148350989879502
235600000 2.0848166271209814
235700000 1.9014577407745918
235800000 2.560668144152516
235900000 3.0381889583861525
236000000 3.169877424461987
236100000 2.0088996355112925
236200000 2.2719308889187775
236300000 2.191501875267635
23

261600000 1.7341461146531438
261700000 3.10923972676911
261800000 3.2883038975140684
261900000 3.135722485963166
262000000 3.1534251654435694
262100000 3.250185064143585
262200000 3.0017318652127423
262300000 3.21956611455675
262400000 3.741171936967558
262500000 3.781625735117002
262600000 3.4489419763175952
262700000 3.0745938396080468
262800000 3.12702074195633
262900000 3.272897867851943
263000000 3.196562673812979
263100000 2.9917211777587864
263200000 3.44305763895759
263300000 0.7329203713744746
263400000 0.7311328526797494
263500000 2.7396042287647524
263600000 3.259129885963331
263700000 3.181772662124981
263800000 3.3952105479047296
263900000 3.162650463150501
264000000 2.921647962057299
264100000 3.2963895646397474
264200000 3.1902585170734485
264300000 4.334920923961027
264400000 4.142446482076526
264500000 3.5285444340137007
264600000 3.3905711544345487
264700000 3.6365230445344747
264800000 3.7564504696792627
264900000 3.4743141566467415
265000000 3.1124213819920223
26510

290300000 2.36384369890864
290400000 2.5057128074072565
290500000 2.136918944910864
290600000 2.1491524942125046
290700000 1.9464961328168313
290800000 2.178450532844692
290900000 2.0533476102800963
291000000 2.019411288703044
291100000 2.123299106101632
291200000 0.8081377037984562
291300000 0.500305533423331
291400000 1.7981937982055245
291500000 1.8985163335149686
291600000 2.024692138564216
291700000 2.161962319579333
291800000 2.1859355914119583
291900000 1.91306671230843
292000000 2.1235546999047
292100000 2.2415942438405057
292200000 2.5881813721896743
292300000 2.1148876081306733
292400000 2.169748854316473
292500000 1.8605640655042197
292600000 2.0312324272814095
292700000 2.1084701753404276
292800000 2.124666875624263
292900000 1.926075076999257
293000000 1.7470005082625613
293100000 0.48175729953485397
293200000 0.4815855399844647
293300000 2.3424209101313322
293400000 1.929180194882638
293500000 2.0447946512453554
293600000 2.0275218179300394
293700000 2.1215113355756237
29

318100000 1.2478596539376676
318200000 1.2413392554741636
318300000 1.2594124986982411
318400000 1.2264493830562333
318500000 1.1146302038710383
318600000 1.360669835774296
318700000 1.1974122591680585
318800000 1.230101817652742
318900000 1.1020589592698449
319000000 1.1298546207305689
319100000 1.1698176155800324
319200000 1.368545570672128
319300000 1.1936867180708746
319400000 1.2323338678213325
319500000 1.1265537487541504
319600000 1.0799506237154304
319700000 1.1523426820175218
319800000 1.290255964906338
319900000 1.1832052126851877
320000000 1.1561447355431715
320100000 1.0842813890631398
320200000 1.1195210396016737
320300000 1.2032545571906932
320400000 1.421576845098214
320500000 1.207415850388348
320600000 1.2041295996234682
320700000 1.1290368819168244
320800000 1.0973157493750156
320900000 1.1254975807919172
321000000 1.1220644926653172
321100000 1.147843766792486
321200000 1.1214324421489603
321300000 1.1012537487408154
321400000 1.0203067295807002
321500000 1.012223320

345100000 0.2375763452863693
345200000 0.1672834607236286
345300000 0.22659890153819987
345400000 0.16174927636232972
345500000 0.21412957014555734
345600000 0.16778215857503812
345700000 0.18900370388254523
345800000 0.17059067175149917
345900000 0.16364194110110736
346000000 0.16558593174115815
346100000 0.13224670627709892
346200000 0.17060553397690587
346300000 0.1305613438014289
346400000 0.16814369133151905
346500000 0.13998032111649047
346600000 0.14999802898710968
346700000 0.1488017524802453
346800000 0.12129609122180277
346900000 0.14029363394458097
347000000 0.12355231852181753
347100000 0.13852669596632322
347200000 0.13289842618447542
347300000 0.1306614721169571
best so far: 0
type: [1, 3, 1, 41, 1] 123
cases of this type: 206763
347400000 0.07447065069725778
347500000 0.018367377747148275
best so far: 0
type: [1, 3, 41, 1, 1] 123
cases of this type: 5043
best so far: 0
type: [1, 41, 1, 1, 3] 123
cases of this type: 3321
best so far: 0
type: [1, 41, 1, 3, 1] 123
cases of 